# SI 330: Data Manipulation
## 05 - Aggregation and Grouping

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.


## Learning Objectives
* use StringIO to create a DataFrame
* use the .describe() function
* understand .groupby()
* know how to use pivot and pivot_table

### IMPORTANT: Replace ```?``` in the following code with your uniqname.

In [39]:
MY_UNIQNAME = '?'

## Before we start...
### <font color="magenta">Q1: (1 point) Please let us know what you found confusing in the last class. </font>
We'll try to take time in the next class to review these concepts next class.


Replace this with your response.

In [40]:
import pandas as pd
import numpy as np

## Some more basic pandas functionality
One of the nice things about pandas is that it simplifies many common operations on datasets.  Let's load yet another LOTR dataset, 
this time using StringIO.  StringIO allows us to create a string that's then available as a file!

Why would you want to do this?  Sometimes it's easier to just paste data right into a Jupyter notebook (or python script)
than it is to create another CSV file.


In [41]:
from io import StringIO

LOTRDATA=StringIO("""name,race,gender,magic,aggression,stealth
Gandalf,Maia,Male,10.0,7.0,8.0
Gimli,Dwarf,None,1.0,10.0,2.0
Frodo,Hobbit,Male,4.0,2.0,5.0
Legolas,Elf,Male,6.0,5.0,10.0
Bilbo,Hobbit,Male,4.0,1.0,5.0
Sam,Hobbit,Male,2.0,6.0,4.0
Pippin,Hobbit,Male,0.0,3.0,5.0
Boromir,Human,Male,0.0,8.0,3.0
Aragorn,Human,Male,2.0,7.0,9.0
Galadriel,Elf,Female,9.0,2.0,10.0
Lily,Hobbit,Female,,,
Meriadoc,Hobbit,Male,,4.0,6.0
Melian,Maia,Female,10.0,5.0,9.0
Idril,Elf,Female,8.0,,8.0
""")

lotr = pd.read_csv(LOTRDATA, index_col=None)
lotr

,name,race,gender,magic,aggression,stealth
0,Gandalf,Maia,Male,10.0,7.0,8.0
1,Gimli,Dwarf,None,1.0,10.0,2.0
2,Frodo,Hobbit,Male,4.0,2.0,5.0
3,Legolas,Elf,Male,6.0,5.0,10.0
4,Bilbo,Hobbit,Male,4.0,1.0,5.0
5,Sam,Hobbit,Male,2.0,6.0,4.0
6,Pippin,Hobbit,Male,0.0,3.0,5.0
7,Boromir,Human,Male,0.0,8.0,3.0
8,Aragorn,Human,Male,2.0,7.0,9.0
9,Galadriel,Elf,Female,9.0,2.0,10.0


Now that we have a DataFrame, we can get some basic statistics about it using the ```describe()``` function.
Note that ```describe()``` only returns values for numeric columns.  Note too that it returns another DataFrame

In [42]:
lotr.describe()

,magic,aggression,stealth
count,12.000000,12.000000,13.000000
mean,4.666667,5.000000,6.461538
std,3.821788,2.730301,2.696151
min,0.000000,1.000000,2.000000
25%,1.750000,2.750000,5.000000
50%,4.000000,5.000000,6.000000
75%,8.250000,7.000000,9.000000
max,10.000000,10.000000,10.000000


In [43]:
type(lotr.describe())

pandas.core.frame.DataFrame

## Pivots/Stack/Unstack Intro
The following cells are based on: 
http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/ , which is
one of the best guides to pivots, pivot tables, stacking and unstacking that I've encountered.

For demonstration purposes, let's create the same DataFrame that Nikolay Grozev uses in his tutorial:

In [44]:
from collections import OrderedDict
from pandas import DataFrame

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)
metal

# create the table

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€


In [45]:
# make a table of items (rows) and costs (USD) 
# for each in gold and bronze
metal.pivot(index='Item',columns='CType',values='USD')



CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$


See the image below... we're telling Pandas to take the table above, create a row for every item. 
This is done by setting index to Item (the column in the original table that contains item names)
We then are telling pandas we want to create a column for every unique element in the
original CType column.  And finally, we want the value in the cells to be the value from the USD
column in the original table.

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple1.png "pivots")

### <font color="magenta">Q2: (2 points) Using pivot and the dataframe below make a table with each person as the index, their purchased items as columns, and the price they paid as the values.</font>


In [46]:
table1 = OrderedDict((
    ("Person", ['Frankie', 'Dr. Teplovs', 'Johan', 'Dr. Teplovs','Frankie','Dr. Teplovs', 'Johan', 'Dr. Teplovs','Frankie', 'Dr. Teplovs']),
    ('Item Purchased',['Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher','Apple', 'Watermelon', 'Pineapple', 'Snickers','Pear']),
    ('Price Paid',  ['$1','$2', '$1.5', '$2', '$1.5','$1','$1','$2', '$1.5', '$2']),
    ))
fav = DataFrame(table1)
fav

,Person,Item Purchased,Price Paid
0,Frankie,Kit-Kat,$1
1,Dr. Teplovs,Mango,$2
2,Johan,Twix,$1.5
3,Dr. Teplovs,M&Ms,$2
4,Frankie,Ferrero Rocher,$1.5
5,Dr. Teplovs,Apple,$1
6,Johan,Watermelon,$1
7,Dr. Teplovs,Pineapple,$2
8,Frankie,Snickers,$1.5
9,Dr. Teplovs,Pear,$2


In [47]:
fav.pivot(index='Person',columns='Item Purchased',values='Price Paid') #Insert Code Here

Item Purchased,Apple,Ferrero Rocher,Kit-Kat,M&Ms,Mango,Pear,Pineapple,Snickers,Twix,Watermelon
Person,,,,,,,,,,
Dr. Teplovs,$1,NaN,NaN,$2,$2,$2,$2,NaN,NaN,NaN
Frankie,NaN,$1.5,$1,NaN,NaN,NaN,NaN,$1.5,NaN,NaN
Johan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$1.5,$1


In [48]:
# end of exercise

In [49]:
p = metal.pivot(index='Item',columns='CType',values='USD')  # same as above, I'm just remembering the pivot as "p"

Access the USD cost of Item0 for Gold customers...

First we find the row for Item0/Gold and then we select the USD column and pull out the value

In [50]:
result = metal[((metal.Item == 'Item0') & (metal.CType == 'Gold'))].USD

In [51]:
type(result)

pandas.core.series.Series

In [52]:
result

0    1$
Name: USD, dtype: object

Do the same thing on pivoted table. Here we pull out the row for Item0, grab the Gold column and print the value

In [53]:
p[p.index == 'Item0'].Gold.values

array(['1$'], dtype=object)

Now pivot by multiple columns, I want USD and EU prices

In [54]:
metal.pivot(index='Item',columns='CType')

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0     2$   1$    NaN     2€   1€    NaN
Item1    NaN   3$     4$    NaN   3€     4€

Access the USD cost of Item0 for Gold customers

In [55]:
p = metal.pivot(index='Item',columns='CType')
p.USD[p.USD.index == 'Item0'].Gold.values

array(['1$'], dtype=object)

# What happens if there is a collision? 
See the problem?  There are two Item0/Golds:


![pivot 2](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple_error.png "pivots")

Let's set up another DataFrame to demonstrate this:

In [56]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)


The next cell will generate an error:

In [57]:
p = metal.pivot(index='Item', columns='CType', values='USD')
# will return an error

ValueError: Index contains duplicate entries, cannot reshape

## pivot_tables is your friend
![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_table_simple1.png "pivots")

Let's create yet another DataFrame to play with:

In [ ]:

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1.1, 2.2, 3.3, 4.4])
))
metal = DataFrame(table)
metal

pivot_table is  a bit different than pivot... It's the same with the first part
index, columns, values remain the same as before BUT we added a rule (aggfunc)
that says: whey you hit a conflict, the way to resolve it is X (in this case
x is the "mean"... so find the mean of the two numbers)

In [ ]:
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.mean)
p

You could have also resolved the conflict in other ways.  Here we tell it to take the "min":

In [ ]:
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.min)
p

### <font color="magenta">Q3: (2 points) Using pivot_table and the dataframe below make a table with each person as the index, their purchased items as columns, and the sum of the price they paid as the values (sometimes people bought more than one of the same item).</font>


In [ ]:
table1 = OrderedDict((
    ("Person", ['Frankie', 'Dr. Teplovs', 'Johan', 'Dr. Teplovs',
                'Frankie','Dr. Teplovs', 'Johan', 'Dr. Teplovs','Frankie', 
                'Dr. Teplovs','Frankie', 'Dr. Teplovs', 'Johan', 'Dr. Teplovs',
                'Frankie','Dr. Teplovs',]),
    ('Item Purchased',['Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher',
                       'Apple', 'Watermelon', 'Pineapple', 'Snickers','Pear','Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher',
                       'Apple']),
    ('Price Paid (USD)',  [1,2, 1.5, 2, 1.5,1,1,2, 1.5, 2,
                    1,2, 1.5, 2, 1.5,1]),
    ))
fav = DataFrame(table1)
fav

In [ ]:
fav.pivot_table(index='Person',columns='Item Purchased',values='Price Paid (USD)',aggfunc=np.sum) #Insert Code Here

Pivots are a specific form of stack/unstack (remember those?)

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/stack-unstack1.png "pivots")

## A worked example

In [58]:
# to start let's make a fake dataset: sales of fruit across US states.
# Don't worry about the details here, but basically we'll pretend
# this string is a CSV file and use the standard loading ops
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales
MI,Walmart,Apple,100
MI,Wholefoods,Apple,150
MI,Kroger,Orange,180
CA,Walmart,Apple,220
CA,Wholefoods,Apple,180
CA,Safeway,Apple,220
CA,Safeway,Orange,110
NY,Walmart,Apple,90
NY,Walmart,Orange,80
NY,Wholefoods,Orange,120
""")

fruit = pd.read_csv(TESTDATA, index_col=None)
fruit

,State,Retailer,Fruit,Sales
0,MI,Walmart,Apple,100
1,MI,Wholefoods,Apple,150
2,MI,Kroger,Orange,180
3,CA,Walmart,Apple,220
4,CA,Wholefoods,Apple,180
5,CA,Safeway,Apple,220
6,CA,Safeway,Orange,110
7,NY,Walmart,Apple,90
8,NY,Walmart,Orange,80
9,NY,Wholefoods,Orange,120


## (a) What is the total sales for each state?
This requires us to group by state, and aggregate sales by taking the sum.

The easiest way of doing this if to use `groupby`

If you execute groupby on the dataframe what you'll get back is an object called DataFrameGroupBy

In [59]:
fruit.groupby('State')

On its own it's a bit useless... it just keeps track of which rows should go into each "pile" (where pile here means a unique group for each state)

If we ask this object to describe itself, you can see what is inside notice that it threw away all the other columns because they were not numerical.  Only "Sales" which is a number, was kept

In [60]:
fruit.groupby('State').describe()

Sales                                                          
      count        mean        std    min    25%    50%    75%    max
State                                                                
CA      4.0  182.500000  51.881275  110.0  162.5  200.0  220.0  220.0
MI      3.0  143.333333  40.414519  100.0  125.0  150.0  165.0  180.0
NY      3.0   96.666667  20.816660   80.0   85.0   90.0  105.0  120.0

Now, if we had another numerical column, let's call it "Sales2," that column would also be kept.  Let's make a fruit2 DataFrame so you can see that:

In [25]:
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales,Sales2
MI,Walmart,Apple,100,10
MI,Wholefoods,Apple,150,20
MI,Kroger,Orange,180,30
CA,Walmart,Apple,220,20
CA,Wholefoods,Apple,180,40
CA,Safeway,Apple,220,30
CA,Safeway,Orange,110,20
NY,Walmart,Apple,90,40
NY,Walmart,Orange,80,20
NY,Wholefoods,Orange,120,60
""")

fruit2 = pd.read_csv(TESTDATA, index_col=None)
fruit2

,State,Retailer,Fruit,Sales,Sales2
0,MI,Walmart,Apple,100,10
1,MI,Wholefoods,Apple,150,20
2,MI,Kroger,Orange,180,30
3,CA,Walmart,Apple,220,20
4,CA,Wholefoods,Apple,180,40
5,CA,Safeway,Apple,220,30
6,CA,Safeway,Orange,110,20
7,NY,Walmart,Apple,90,40
8,NY,Walmart,Orange,80,20
9,NY,Wholefoods,Orange,120,60


In [26]:
fruit2.groupby("State").describe()

Sales                                                           Sales2  \
      count        mean        std    min    25%    50%    75%    max  count   
State                                                                          
CA      4.0  182.500000  51.881275  110.0  162.5  200.0  220.0  220.0    4.0   
MI      3.0  143.333333  40.414519  100.0  125.0  150.0  165.0  180.0    3.0   
NY      3.0   96.666667  20.816660   80.0   85.0   90.0  105.0  120.0    3.0   

                                                      
       mean        std   min   25%   50%   75%   max  
State                                                 
CA     27.5   9.574271  20.0  20.0  25.0  32.5  40.0  
MI     20.0  10.000000  10.0  15.0  20.0  25.0  30.0  
NY     40.0  20.000000  20.0  30.0  40.0  50.0  60.0

To actually make use of the groupby, we need to tell pandas what to use to measure what's in each group. In other words, I've created a pile for California, a pile for Michigan, and a pile for New York.  I want a number to what's *inside* each pile.  I could ask for the "size" (so how many rows are in each pile), or I could calculate some mathematical function.  For example, if I wanted to know the total sales, I would call "sum."  What happens is pandas goes through every pile, looks at every "row" inside that pile and, for all numerical properties, calculated something.  In this case it's sum... it adds up everything.  So in our original table we had three items for Michigan (Walmart, Kroger, Wholefoods).  This is our Michigan pile.  We then look at numerical properties for Walmart, Kroger, and Wholefoods. In this case Sales.  Because we are using sum() that means add the sales of each.

In [27]:
# What are the total sales for each state?
fruit.groupby('State').sum()  # instead of size()

,Sales
State,
CA,730
MI,430
NY,290


What just happend? A couple of things:
- `groupby()` got first executed on `df`, returning an `DataFrameGroupBy` object. This object itself is useless unless coupled with an aggregation function, such as `sum()`, `mean()`, `max()`, `apply()`. We will talk about `apply()` more in the next week.
- Then, `sum()` got executed on the `DataFrameGroupBy` object, generating the `DataFrame` object you see above. Notice how the table looks different than the original DataFrame `df`? Here are the differences:
  - The `State` column now becomes the index of the DataFrame. The string "State" is the name of the index. Notice how the index name is displayed on a lower level than column names.
  - Since we performed a `groupby` operation by `State`, so only the unique values of `State` are kept as index.
  - Among the other columns, Retailer, Fruit, and Sales, only Sales is kept in the result table. This is because the aggregation function `sum()` only knows how to aggregate numerical values. And only Sales is a numerical column. The other columns are hence dropped.

### <font color="magenta">Q4: (2 points) Using groupby, which Retailer had the highest total Sales and how much was that:

In [64]:
fruit.groupby('Retailer').sum() # Insert code here

,Sales
Retailer,
Kroger,180
Safeway,330
Walmart,490
Wholefoods,450


Insert written Answer here

## (b) What is the total sales for each state for each fruit?
This requires us to perform `groupby` on two columns. So, we provide a list of column names to the `groupby` function.

Don't forget that an aggregation function needs to follow the `groupby` function in order to generate results.

In [28]:
# What is the total sales for each state for each fruit?
fruit.groupby(['State','Fruit']).sum()

Sales
State Fruit        
CA    Apple     620
      Orange    110
MI    Apple     250
      Orange    180
NY    Apple      90
      Orange    200

How is this DataFrame different from the previous one?

The biggest different is that this DataFrame has what is called a `MultiIndex` (or hierarchical index), as opposed to a simple index. In this table, the left two "columns" are not columns but actually part of the `MultiIndex`, and the `Sales` is the single real "column" in the DataFrame. (Running out of terminologies here...)

The hierarchical index can be organized in an alternative way if we swapped the order of State and Fruit.

In [29]:
fruit.groupby(['Fruit','State']).sum()

Sales
Fruit  State       
Apple  CA       620
       MI       250
       NY        90
Orange CA       110
       MI       180
       NY       200

## (c) Which state has the maximum total sales?
This question is not asking about the maximum value, but rather which state holds that maximum. There are multiple ways to do it. A principled way is to use `idxmax`.

In [30]:
# Which state has the maximum total sales?
fruitSalesByState = fruit.groupby('State').sum()
print(fruitSalesByState)
max_state = fruitSalesByState['Sales'].idxmax()
print("The state with the maximum sales is: ",max_state)

       Sales
State       
CA       730
MI       430
NY       290
The state with the maximum sales is:  CA




What if I want to display the maximum value alongside the state? Well, we can use that returned label to _select_ the corresponding row from the original DataFrame.

In [31]:
# the deets
fruitSalesByState.loc['CA']

Sales    730
Name: CA, dtype: int64

A less efficient but more intuitive way of doing the same thing:

In [32]:
fruitSalesByState.sort_values('Sales',ascending=False).iloc[0]

Sales    730
Name: CA, dtype: int64

### Which state has the maximum total sales for apples?</font>

In [147]:
# Which state has the maximum total sales for apples?
# give me apple sellers
apples = fruit[fruit.Fruit == 'Apple']
# aggr. by state
applesByState = apples.groupby('State').sum()
applesByState

,Sales
State,
CA,620
MI,250
NY,90


In [150]:
applesByState.Sales

State
CA    620
MI    250
NY     90
Name: Sales, dtype: int64

In [151]:
applesByState.Sales.idxmax()

'CA'

In [183]:
applesByState.loc[applesByState.Sales.idxmax()]

Sales    620
Name: CA, dtype: int64

In the above command, `.loc[]` looks up the index label and returns that row.



## Applying what we learned to the LOTR data

What are the average values for magic, aggression, and stealth for each race?

In [184]:
lotr.groupby('race').mean()

,magic,aggression,stealth
race,,,
Dwarf,1.000000,10.0,2.000000
Elf,7.666667,3.5,9.333333
Hobbit,2.500000,3.2,5.000000
Human,1.000000,7.5,6.000000
Maia,10.000000,6.0,8.500000


We can make things a bit more complex by creating a dictionary of aggregations.  
The keys of the dictionary refer to the originating column we want to transform;
the values of the dictionary contain lists of the functions we want to apply to those columns.

Note that we can use built-in functions (see Table 10-1 in py4da) or define our own, possibly with a lambda:

In [190]:
aggregations = {
    'magic': [min, max, sum, len],
    'aggression': [min, max, sum, len],
    'stealth': [min, max, lambda x: max(x) - min(x)]
}

In [193]:
grouped = lotr.groupby('race').agg(aggregations)
grouped

magic                  aggression                  stealth        \
         min   max   sum  len        min   max   sum  len     min   max   
race                                                                      
Dwarf    1.0   1.0   1.0  1.0       10.0  10.0  10.0  1.0     2.0   2.0   
Elf      6.0   9.0  23.0  3.0        2.0   5.0   7.0  3.0     8.0  10.0   
Hobbit   0.0   4.0  10.0  6.0        1.0   6.0  16.0  6.0     4.0   6.0   
Human    0.0   2.0   2.0  2.0        7.0   8.0  15.0  2.0     3.0   9.0   
Maia    10.0  10.0  20.0  2.0        5.0   7.0  12.0  2.0     8.0   9.0   

                 
       <lambda>  
race             
Dwarf       0.0  
Elf         2.0  
Hobbit      2.0  
Human       6.0  
Maia        1.0

In [194]:
grouped.columns

MultiIndex(levels=[['magic', 'aggression', 'stealth'], ['<lambda>', 'len', 'max', 'min', 'sum']],
           labels=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2], [3, 2, 4, 1, 3, 2, 4, 1, 3, 2, 0]])

In [195]:
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]


In [196]:
grouped.columns.ravel()

array(['magic_min', 'magic_max', 'magic_sum', 'magic_len',
       'aggression_min', 'aggression_max', 'aggression_sum',
       'aggression_len', 'stealth_min', 'stealth_max', 'stealth_<lambda>'],
      dtype=object)

In [197]:
grouped

,magic_min,magic_max,magic_sum,magic_len,aggression_min,aggression_max,aggression_sum,aggression_len,stealth_min,stealth_max,stealth_<lambda>
race,,,,,,,,,,,
Dwarf,1.0,1.0,1.0,1.0,10.0,10.0,10.0,1.0,2.0,2.0,0.0
Elf,6.0,9.0,23.0,3.0,2.0,5.0,7.0,3.0,8.0,10.0,2.0
Hobbit,0.0,4.0,10.0,6.0,1.0,6.0,16.0,6.0,4.0,6.0,2.0
Human,0.0,2.0,2.0,2.0,7.0,8.0,15.0,2.0,3.0,9.0,6.0
Maia,10.0,10.0,20.0,2.0,5.0,7.0,12.0,2.0,8.0,9.0,1.0


### Which race has the lowest average magic value?

In [198]:
lotr.groupby('race')['magic'].mean().idxmin()

'Dwarf'

Remind ourselves what our DataFrame looks like:

In [199]:
lotr

,name,race,gender,magic,aggression,stealth
0,Gandalf,Maia,Male,10.0,7.0,8.0
1,Gimli,Dwarf,None,1.0,10.0,2.0
2,Frodo,Hobbit,Male,4.0,2.0,5.0
3,Legolas,Elf,Male,6.0,5.0,10.0
4,Bilbo,Hobbit,Male,4.0,1.0,5.0
5,Sam,Hobbit,Male,2.0,6.0,4.0
6,Pippin,Hobbit,Male,0.0,3.0,5.0
7,Boromir,Human,Male,0.0,8.0,3.0
8,Aragorn,Human,Male,2.0,7.0,9.0
9,Galadriel,Elf,Female,9.0,2.0,10.0


### <font color="magenta">Q5: (3 points) Create a pivot table showing the maximum values of magic using gender for columns and race for rows.  

In [65]:
lotr.pivot_table(index='race',columns='gender',values='magic',dropna=False,aggfunc='min') #Insert Code Here

gender,Female,Male,None
race,,,
Dwarf,NaN,NaN,1.0
Elf,8.0,6.0,NaN
Hobbit,NaN,0.0,NaN
Human,NaN,0.0,NaN
Maia,10.0,10.0,NaN
